In [8]:
#Python 3.11.9

In [10]:
#pip install findspark

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [14]:
#pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Pagerank with graph Projections") \
        .master("yarn") \
        .config("spark.executor.instances", "3") \
        .config("spark.executor.memory", "6g") \
        .config("spark.driver.memory", "4g") \
        .config("spark.locality.wait.node", "0") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.jars.packages", "graphframes:graphframes:0.8.4-spark3.5-s_2.12") \
        .getOrCreate()
# .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.1-s_2.12") \
# .config("spark.hadoop.fs.DefaultFS", "hdfs://localhost:9000") \
#        .config("spark.memory.storageFraction" , "0.3") \
#        


:: loading settings :: url = jar:file:/home/master/spark-3.5.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/master/.ivy2/cache
The jars for the packages stored in: /home/master/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f212c871-194f-4dae-b9dc-cc256ea72fc8;1.0
	confs: [default]
	found graphframes#graphframes;0.8.4-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 217ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.4-spark3.5-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

In [4]:
# Test the spark session
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.4


In [5]:
spark.sparkContext._conf.getAll()  # check the config

[('spark.yarn.secondary.jars',
  'graphframes_graphframes-0.8.4-spark3.5-s_2.12.jar,org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.executor.instances', '3'),
 ('spark.driver.appUIAddress', 'http://master:4040'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://master:8088/proxy/application_1736335349974_0003'),
 ('spark.app.startTime', '1736342949260'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'master'),
 ('spark.driver.memory', '4g'),
 ('spark.executorEnv.PYTHONPATH',
  '{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.7-src.zip<CPS>{{PWD}}/graphframes_graphframes-0.8.4-spark3.5-s_2.12.jar<CPS>{{PWD}}/org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.locality.wait.node', '0'),
 ('spark.app.submitTime', '1736342948942'),
 ('spark.yarn.dist.pyFiles',
  'file:///home/master/.ivy2/jars/graphframes_graphframes-0.8.4-spark3.5-s_2.12.jar,file:///home/master/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'),
 ('sp

In [6]:
hdfs_port = 9000

In [7]:
ratings = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/ratings.csv', header=True, inferSchema=True)

In [8]:
movies = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/movies.csv', header=True, inferSchema=True)

In [9]:
user_movie_matrix = ratings.join(movies, on="movieId", how="inner") # parallizzare

In [10]:
user_movie_matrix.limit(5).toPandas()

,movieId,userId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3,1,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,6,1,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,47,1,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,50,1,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [11]:
mapping_score = {
    0.5: -1.2,
    1: -1.1,
    1.5: -1.0,
    2: -0.5,
    2.5: -0.1,
    3: 0.1,
    3.5: 0.5,
    4: 1.0,
    4.5: 1.1,
    5: 1.2
}

In [12]:
from pyspark.sql.types import IntegerType, FloatType, StringType
from pyspark.sql.functions import col

map_score_udf = spark.udf.register("map_score", lambda x: mapping_score.get(x, 0), FloatType())
user_movie_matrix = user_movie_matrix.withColumn("weight", map_score_udf(col("rating")))

In [13]:
edges = user_movie_matrix.select(
    col("userId").cast(StringType()).alias("src"),
    col("title").alias("dst"),
    col("weight")
)

In [14]:
from pyspark.sql.functions import lit

user_vertices = user_movie_matrix.select(col("userId").cast(StringType()).alias("id")).distinct().withColumn("bipartite", lit(0))
movie_vertices = user_movie_matrix.select(col("title").alias("id")).distinct().withColumn("bipartite", lit(1))
vertices = user_vertices.union(movie_vertices)

In [14]:
#pip install graphframes

From [Stackoverflow](https://stackoverflow.com/questions/39261370/unable-to-run-a-basic-graphframes-example)

$\dots$

Depending on your spark version, all you have to do is download the graphframe jar corresponding to your version
of spark here https://spark-packages.org/package/graphframes/graphframes.

Then you'll have to copy the jar downloaded to your spark jar directory and run

```shell
pyspark --packages graphframes:graphframes-0.8.4-spark3.5-s_2.12 --jars graphframes-0.8.4-spark3.5-s_2.12.jar
```

This is included in the SparkConfig

In [15]:
from graphframes import GraphFrame

user_movie_graph = GraphFrame(vertices, edges)

In [16]:
print("First 10 nodes in the graph:")
user_movie_graph.vertices.show(10, truncate=False)

First 10 nodes in the graph:


[Stage 9:>                                                          (0 + 1) / 1]

+---+---------+
|id |bipartite|
+---+---------+
|296|0        |
|467|0        |
|125|0        |
|451|0        |
|7  |0        |
|51 |0        |
|124|0        |
|447|0        |
|591|0        |
|307|0        |
+---+---------+
only showing top 10 rows



In [17]:
user_nodes = user_movie_graph.vertices.filter(col("bipartite") == 0)
user_count = user_nodes.count()
print(f"Numero di nodi con bipartite = 0 (utenti): {user_count}")
user_nodes.show(10, truncate=False)

Numero di nodi con bipartite = 0 (utenti): 610
Esempio di nodi con bipartite = 0 (utenti):
+---+---------+
|id |bipartite|
+---+---------+
|296|0        |
|467|0        |
|125|0        |
|451|0        |
|7  |0        |
|51 |0        |
|124|0        |
|447|0        |
|591|0        |
|307|0        |
+---+---------+
only showing top 10 rows



In [18]:
movie_nodes = user_movie_graph.vertices.filter(col("bipartite") == 1)
movie_count = movie_nodes.count()
print(f"Numero di nodi con bipartite = 1 (film): {movie_count}")
movie_nodes.show(10, truncate=False)

Numero di nodi con bipartite = 1 (film): 9719
Esempio di nodi con bipartite = 1 (film):
+--------------------------------------+---------+
|id                                    |bipartite|
+--------------------------------------+---------+
|Psycho (1960)                         |1        |
|Men in Black (a.k.a. MIB) (1997)      |1        |
|Gulliver's Travels (1939)             |1        |
|Heavenly Creatures (1994)             |1        |
|Elizabeth (1998)                      |1        |
|Before Night Falls (2000)             |1        |
|O Brother, Where Art Thou? (2000)     |1        |
|Snow White and the Seven Dwarfs (1937)|1        |
|Three Wishes (1995)                   |1        |
|When We Were Kings (1996)             |1        |
+--------------------------------------+---------+
only showing top 10 rows



In [19]:
num_edges = user_movie_graph.edges.count()
print(f"Number of edges: {num_edges}")

print("First 10 edges in the graph:")
user_movie_graph.edges.show(10, truncate=False)

First 10 edges in the graph:


[Stage 36:>                                                         (0 + 1) / 1]

+---+---------------------------+------+
|src|dst                        |weight|
+---+---------------------------+------+
|1  |Toy Story (1995)           |1.0   |
|1  |Grumpier Old Men (1995)    |1.0   |
|1  |Heat (1995)                |1.0   |
|1  |Seven (a.k.a. Se7en) (1995)|1.2   |
|1  |Usual Suspects, The (1995) |1.2   |
|1  |From Dusk Till Dawn (1996) |0.1   |
|1  |Bottle Rocket (1996)       |1.2   |
|1  |Braveheart (1995)          |1.0   |
|1  |Rob Roy (1995)             |1.2   |
|1  |Canadian Bacon (1995)      |1.2   |
+---+---------------------------+------+
only showing top 10 rows



In [20]:
# Check the memory occupation
vertices_size = user_movie_graph.vertices.rdd.map(lambda x: len(str(x))).sum()
edges_size = user_movie_graph.edges.rdd.map(lambda x: len(str(x))).sum()

print(f"Estimated memory usage for vertices: {vertices_size/(1024**2)} MB")
print(f"Estimated memory usage for edges: {edges_size/(1024**2)} MB")

[Stage 45:>                                                         (0 + 1) / 1]

Estimated memory usage for vertices: 0.46789073944091797 MB
Estimated memory usage for edges: 6.4138898849487305 MB


In [22]:
movie_nodes = user_movie_graph.vertices.filter(col("bipartite") == 1).select("id").alias("movies")
print(f"Number of nodes{movie_nodes.count()}")

In Python the [projection algoritm](https://networkx.org/documentation/stable/_modules/networkx/algorithms/bipartite/projection.html#weighted_projected_graph) is implemented as follows:

```python
def weighted_projected_graph(B, nodes, ratio=False):
    
    if B.is_directed():
        pred = B.pred
        G = nx.DiGraph()
    else:
        pred = B.adj
        G = nx.Graph()
    G.graph.update(B.graph)
    G.add_nodes_from((n, B.nodes[n]) for n in nodes)
    n_top = len(B) - len(nodes)

    if n_top < 1:
        raise NetworkXAlgorithmError(
            f"the size of the nodes to project onto ({len(nodes)}) is >= the graph size ({len(B)}).\n"
            "They are either not a valid bipartite partition or contain duplicates"
        )

    for u in nodes:
        unbrs = set(B[u])
        nbrs2 = {n for nbr in unbrs for n in B[nbr]} - {u}
        for v in nbrs2:
            vnbrs = set(pred[v])
            common = unbrs & vnbrs
            if not ratio:
                weight = len(common)
            else:
                weight = len(common) / n_top
            G.add_edge(u, v, weight=weight)
    return G
```

In [25]:
from pyspark.sql.functions import col, count, size, collect_set, array_intersect

def weighted_projected_graph(edges, movies, ratio=False):
    movie_neighbors = edges.groupBy("src").agg(collect_set("dst").alias("neighbors"))
    movie_neighbors = movie_neighbors.alias("mn1").join(movie_neighbors.alias("mn2"), col("mn1.src") < col("mn2.src")) \
        .select(
            col("mn1.src").alias("movie1"),
            col("mn2.src").alias("movie2"),
            array_intersect(col("mn1.neighbors"), col("mn2.neighbors")).alias("shared_neighbors")
        )

    if not ratio:
        movie_neighbors = movie_neighbors.withColumn("weight", size(col("shared_neighbors")))
    else:
        n_top = edges.select("src").distinct().count()
        movie_neighbors = movie_neighbors.withColumn("weight", size(col("shared_neighbors")) / n_top)
    
    movie_neighbors = movie_neighbors.filter(col("weight") > 0)

    movie_movie_edges = movie_neighbors.select(
        col("movie1").alias("src"),
        col("movie2").alias("dst"),
        col("weight")
    )
    
    movie_movie_graph = GraphFrame(movies, movie_movie_edges)
    return movie_movie_graph

In [26]:
movie_movie_graph = weighted_projected_graph(user_movie_graph.edges, movie_nodes, ratio=False)

[Stage 73:=============================>                            (1 + 1) / 2]

Number of vertices (nodes): 9719
Number of edges: 164054


In [ ]:
num_vertices = movie_movie_graph.vertices.count()
num_edges = movie_movie_graph.edges.count()
print(f"Number of vertices (nodes): {num_vertices}")
print(f"Number of edges: {num_edges}")

In [28]:
print("First 10 edges in the graph:")
movie_movie_graph.vertices.show(10, truncate=False)

First 10 edges in the graph:
+--------------------------------------+
|id                                    |
+--------------------------------------+
|Psycho (1960)                         |
|Men in Black (a.k.a. MIB) (1997)      |
|Gulliver's Travels (1939)             |
|Heavenly Creatures (1994)             |
|Elizabeth (1998)                      |
|Before Night Falls (2000)             |
|O Brother, Where Art Thou? (2000)     |
|Snow White and the Seven Dwarfs (1937)|
|Three Wishes (1995)                   |
|When We Were Kings (1996)             |
+--------------------------------------+
only showing top 10 rows



In [27]:
print("First 10 edges in the graph:")
movie_movie_graph.edges.show(10, truncate=False)

First 10 edges in the graph:
+---+---+------+
|src|dst|weight|
+---+---+------+
|100|101|7     |
|100|102|12    |
|100|104|35    |
|100|107|7     |
|100|11 |10    |
|100|110|3     |
|100|111|22    |
|100|112|14    |
|100|113|23    |
|100|118|7     |
+---+---+------+
only showing top 10 rows



In [30]:
# Check the memory occupation
vertices_size = movie_movie_graph.vertices.rdd.map(lambda x: len(str(x))).sum()
edges_size = movie_movie_graph.edges.rdd.map(lambda x: len(str(x))).sum()

print(f"Estimated memory usage for vertices: {vertices_size/(1024**2)} MB")
print(f"Estimated memory usage for edges: {edges_size/(1024**2)} MB")

[Stage 100:============================>                            (1 + 1) / 2]

Estimated memory usage for vertices: 0.33237457275390625 MB
Estimated memory usage for edges: 5.493730545043945 MB


In [32]:
def create_preference_vector(user_id, user_movie_graph):
    edges = user_movie_graph.edges.filter(col("src") == user_id).rdd.map(lambda row: (row["dst"], row["weight"])).collect()
    print(f"Preference vector for user {user_id}: {list(edges)[:10]}")
    
    tot = sum([weight for _, weight in edges])
    
    print(f"Total weight for user {user_id}: {tot}")
    if tot > 0:
        return {movie: weight / tot for movie, weight in edges} # <- qua è diverso perchè assegno peso SOLO a quelli visti, in python normale assegnavmo a tutti i film qualcosa
    else:
        movies = user_movie_graph.vertices.filter(col("bipartite") == 1).select("id").rdd.map(lambda row: row[0]).collect()
        return {movie: 1 / len(movies) for movie in movies}

In [56]:
user = "10"
p_vec = create_preference_vector(user, user_movie_graph)
print("The user has rated", len(p_vec), "movies")
already_seen = [movie for movie, weight in p_vec.items() if weight > 0]

Preference vector for user 10: [('Pulp Fiction (1994)', -1.100000023841858), ('Forrest Gump (1994)', 0.5), ('Aladdin (1992)', 1.0), ('Pretty Woman (1990)', 0.5), ('Casablanca (1942)', 1.0), ('Mary Poppins (1964)', -1.2000000476837158), ('Dirty Dancing (1987)', 0.10000000149011612), ('Graduate, The (1967)', 0.10000000149011612), ('When Harry Met Sally... (1989)', 0.10000000149011612), ('As Good as It Gets (1997)', 0.5)]
Total weight for user 10: 50.10000038892031
The user has rated 140 movies


In [42]:
def predict_user(user_id, user_movie_graph, movie_movie_graph):
    p_vec = create_preference_vector(user_id, user_movie_graph)
    print(f"Preference vector for user {user_id}: {list(p_vec)[:10]}")
    already_seen = [movie for movie, weight in p_vec.items() if weight > 0]
    print(f"Already seen movies for user {user_id}: {list(already_seen)[:10]}")
    if len(already_seen) == len(p_vec):
        return []

    temp =  []
    for k, v in p_vec.items():
        temp.append(k)

    pagerank_results = movie_movie_graph.pageRank(resetProbability=0.95, maxIter=20)
    #pagerank_results = movie_movie_graph.pageRank(resetProbability=0.95, maxIter=20, sourceId=p_vec.keys())
    #pagerank_results = movie_movie_graph.parallelPersonalizedPageRank(resetProbability=0.15, maxIter=10, sourceIds=temp)
    
    item_rank = pagerank_results.vertices.select("id", "pageranks").rdd.map(lambda row: (row["id"], row["pageranks"])).collectAsMap()
    
    #
    recommendations = sorted(
        (movie for movie in item_rank if movie not in already_seen),
        key=lambda x: item_rank[x], reverse=True
    )
    #

    return recommendations[:10]

In [ ]:
user = "10"
s_t = predict_user(user, user_movie_graph, movie_movie_graph)
print(f"Predicted movies for user {user}: {s_t}")

In [ ]:
upload_test = False

In [ ]:
# Delete old test
if upload_test:
    def delete_test(tx, userId, movieId):
        tx.run("MATCH (u:User {userId: $userId})-[r:RECOMMENDED]->(m:Movie {movieId: $movieId}) DELETE r", userId=userId, movieId=movieId)

    with driver.session() as session:
        for movieId in s_t:           
            session.execute_write(delete_test, user, movieId)

In [ ]:
# Upload the test
if upload_test:
    def create_recommendations(tx, userId, recs):
            for rec in recs:
                tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
                    "MERGE (u)-[:RECOMMENDED]->(m)",
                    userId=userId, title=rec)

    with driver.session() as session:
        session.execute_write(create_recommendations, user, s_t)

In [ ]:
def create_recommendations(tx, userId, recs):
    for rec in recs:
        tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
            "MERGE (u)-[:RECOMMENDED]->(m)",
            userId=userId, title=rec)
            
if upload_predictions:
    with driver.session() as session:
       for user in filter_nodes(user_movie_graph, 0):
           recs = predict_user(False, user, user_movie_graph, movie_movie_graph)    
           session.execute_write(create_recommendations, user, recs)

In [ ]:
driver.close()
exit()